# How to Build a Crypto Algorithmic Trading Bot in 6 Steps

# Introduction

In this tutorial, we’ll be using Alpaca and Python to create a fully scalable algorithmic crypto trading bot that can execute in your sleep. The strategy that will be used is based on the popular Supertrend indicator.

Without further ado, let’s get started!

# Getting Started

**Download Dependencies**

First, we must pip install (download) the required dependencies for the program. These dependencies are what make it possible to collect and calculate the data we need to build the trading bot.

In [ ]:
%%capture
!pip install alpaca-trade-api

**Import Modules**

We’ll need to import all of the required dependencies we’ll be using in the bot including smtplib for email communication, pandas-ta for the calculation of the supertrend indicator, and alpaca-trade-api for live market data and paper trading account access.

In [ ]:
# Import Dependencies
import smtplib
import pandas as pd
import pandas_ta as ta
import alpaca_trade_api as tradeapi
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

**Setup REST API and Declare Variables**

The next step is to define most of the general variables we’ll be needing throughout the program. For the API and Secret keys, you can paste the values you stored from step 1 of this tutorial when creating an Alpaca account. 

For the trading strategy, we’ll be buying and selling Bitcoin and each transaction will consist of 1 quantity.

In [ ]:
# API Credentials
API_KEY = 'APCA-API-KEY-ID'
SECRET_KEY = 'APCA-API-SECRET-KEY'
api = tradeapi.REST(API_KEY, SECRET_KEY,'https://paper-api.alpaca.markets')

# Define crypto related variables
symbol = 'BTCUSD'
qty_per_trade = 1

**Create Function to Check Account Positions**

Next, we have to create a function to check whether the trading account currently holds any Bitcoin. If it does, we can return the quantity of Bitcoin that is being held. If there is no Bitcoin, then the function will just return 0.

This is important because, in the next function which handles the buying and selling, we can focus on buying only if there is currently no Bitcoin in the account. Otherwise, we’ll monitor the sell signal to see if the position should be closed.

In [ ]:
# Check Whether Account Currently Holds Symbol
def check_positions(symbol):
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0

**Create Function to Trade Bitcoin**

Finally, we can create a function to retrieve the data, calculate the supertrend indicator signals, and execute the buy/sell orders. First, the function takes in an input of bar data which then gets converted into a dataframe to be further manipulated. Using the pandas-ta module discussed earlier, we can calculate the supertrend indicator as a column and concatenate it to the dataframe with pricing information.

If the closing price is above the supertrend indicator, then a buy signal is issued. If the closing price is below the supertrend indicator, then a sell signal is issued. So by using the latest values from the dataframe, we can create boolean values for buy and sell signals.

We can set the code to buy 1 quantity of Bitcoin if there is currently no position and if a buy signal is true. Likewise, we can set the code to sell 1 quantity of Bitcoin if there is currently a position and if a sell signal is true.

Wrapping the entire function in a try/except block ensures that the program will not break due to errors, and will simply print out the error message. Since this is a trading bot and is intended to run throughout market hours, it’s best if the program is continuously running.

In [ ]:
# Supertrend Indicator Bot Function
def supertrend_bot(bar):
    try:
        # Get the Latest Data
        dataframe = api.get_crypto_bars(symbol, tradeapi.TimeFrame(1, tradeapi.TimeFrameUnit.Minute)).df
        dataframe = dataframe[dataframe.exchange == 'CBSE']
        sti = ta.supertrend(dataframe['high'], dataframe['low'], dataframe['close'], 7, 3)
        dataframe = pd.concat([dataframe, sti], axis=1)

        position = check_positions(symbol=symbol)
        should_buy = bar['c'] > dataframe["SUPERT_7_3.0"][-1]
        should_sell = bar['c'] < dataframe["SUPERT_7_3.0"][-1]
        print(f"Price: {dataframe.close[-1]}")
        print("Super Trend Indicator: {}".format(dataframe["SUPERT_7_3.0"][-1]))
        print(f"Position: {position} | Should Buy: {should_buy}")

        # Check if No Position and Buy Signal is True
        if position == 0 and should_buy == True:
            api.submit_order(symbol, qty=qty_per_trade, side='buy')
            message = f'Symbol: {symbol} | Side: Buy | Quantity: {qty_per_trade}'
            print(message)
            send_mail(message)

        # Check if Long Position and Sell Signal is True
        elif position > 0 and should_sell == True:
            api.submit_order(symbol, qty=qty_per_trade, side='sell')
            message = f'Symbol: {symbol} | Side: Sell | Quantity: {qty_per_trade}'
            print(message)
            send_mail(message)
        print("-"*20)

    except Exception as e:
        print (e)
        send_mail(f"Trading Bot failed due to {e}")

**Create Function for Sending Emails**

A critical aspect of any algorithmic trading system is having a built-in communication system where updates can be sent about any potential errors, transactions, or status. With Python, it’s extremely easy to set up email messaging using the packages email and smtplib.
The first step is to set the variables for the sending and receiving emails. The email that is sending the messages must have the setting “Less Secure Apps” set to be off. Due to security reasons, it’s for the best if you create an entirely new email just for this trading strategy.

Next, we can set the “from” tag equal to the algorithm name, subject title equal to our bot, and mail content using the input of text for the function. Using smtplib, we can access the gmail server, log in using the credentials given, and send the email message. You can now uncomment all of the send_mail function calls from the previous step!

In [ ]:
# Send an Update Email After Every Trade
def send_mail(message_text):
    # Define email related variables
    sender_email = '***********'
    sender_password = '***********'
    receiver_email = '***********'

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message['From'] = 'Crypto Trading Algorithm'
    message['To'] = receiver_email
    message['Subject'] = 'Supertrend Indicator Bot'

    # Add body to email and send
    mail_content = message_text
    message.attach(MIMEText(mail_content, 'plain'))
    session = smtplib.SMTP('smtp.gmail.com', 587)
    session.starttls()
    session.login(sender_email, sender_password)
    text = message.as_string()
    session.sendmail(sender_email, receiver_email, text)
    session.quit()
    print ('Mail Sent')

    return {"Success": True}

**Set Up Alpaca Live Crypto Data**

The last step of building the Python bot is to start streaming live market data for Bitcoin from Alpaca. Fortunately, Alpaca makes this process extremely easy. 

First, we have to create an instance of the data streaming API by calling the Stream method in which we pass the API keys. We can also specify that we want raw data only from the Coinbase exchange. Then, we can create an asynchronous function to receive the live bar data and within this function, we can call the supertrend_bot function we made in step 2. 

Here, we can send an email that the trading bot is now live, subscribe to crypto data, and start streaming live data!

In [ ]:
# Create instance of Alpaca data streaming API
alpaca_stream = tradeapi.Stream(API_KEY, SECRET_KEY, raw_data=True, crypto_exchanges=['CBSE'])

# Create handler for receiving live bar data
async def on_crypto_bar(bar):
    print(bar)
    supertrend_bot(bar)

send_mail("Trading Bot is Live")

# Subscribe to data and assign handler
alpaca_stream.subscribe_crypto_bars(on_crypto_bar, symbol)

# Start streaming of data
alpaca_stream.run()